# Fatoração de Matrizes para Sistemas de Classificação de Machine Learning
## CCM0218 2020.2
### Fernando Valls Yoshida (11246714) e Lucas de Sousa Rosa (11296717)

In [4]:
import numpy as np
import math

In [26]:
def rot_Givens(W, n, m, i, j, c, s):
    for r in range(0,m):
        aux = c*W[i,r] - s*W[j,r]
        W[j,r] = s*W[i,r] + c*W[j,r]
        W[i,r] = aux

In [27]:
def sin_cos(W, i, j, k):
    if abs(W[i,k]) > abs(W[j,k]):
        t = -W[j,k]/W[i,k]
        c = 1/math.sqrt(1 + t**2)
        s = c*t
    else:
        t = -W[i,k]/W[j,k]
        s = 1/math.sqrt(1 + t**2)
        c = s*t    
    return s, c

In [28]:
W = np.array([[1,2], [3,4]])
s, c = sin_cos(W, 0, 1, 0)
print(W)
rot_Givens(W, 1, 1, 0, 1, c, s)
print(W)

[[1 2]
 [3 4]]
[[-3  2]
 [ 0  4]]
